<a href="https://colab.research.google.com/github/jjkcoding/Personal-Expenses-Dashboard/blob/main/Spending_Analysis_(Feb2021_Feb2022).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary libraries

In [36]:
import pandas as pd
import numpy as np

Importing excel data into a dataframe

In [13]:
spending_df = pd.read_excel("/content/my_spendings_feb2021_feb2022.xlsx", header = 0)
spending_df

,Date,Description,Spending,Debits,Credits
0,2021-02-01,Check Card: Spotify USA 877-7781161 NY 01/31/21,-4.99,4.99,0.0
1,2021-02-03,From Share 00 REGULAR SAVINGS electric,150.00,0.00,150.0
2,2021-02-03,From Share 00 REGULAR SAVINGS rent,500.00,0.00,500.0
3,2021-02-04,2005LEEABRAZIL-1 TYPE: WEB PMTS CO: 2005LEEAB...,-500.00,500.00,0.0
4,2021-02-04,PGANDE TYPE: WEB ONLINE CO: PGANDE,-134.91,134.91,0.0
...,...,...,...,...,...
287,2022-01-09,Check Card: SQ *TENORI Stanton CA 01/07/22,-9.70,9.70,0.0
288,2022-01-09,Check Card: THE RINKS - ANAHEIM ICE 7145357465...,-6.00,6.00,0.0
289,2022-01-10,Check Card: TARGET 00019364 SANTA ANA CA 01/0...,-17.45,17.45,0.0
290,2022-01-10,Check Card: PANDA EXPRESS #108 SANTA ANA CA 0...,-11.27,11.27,0.0


Filtering out deposits into bank so we only analyze the expenses and dropping unnecessary columns

In [14]:
debt_df = spending_df.loc[spending_df.Debits > 0]
debt_df = debt_df.drop(['Credits', 'Spending'], axis = 1)
debt_df

,Date,Description,Debits
0,2021-02-01,Check Card: Spotify USA 877-7781161 NY 01/31/21,4.99
3,2021-02-04,2005LEEABRAZIL-1 TYPE: WEB PMTS CO: 2005LEEAB...,500.00
4,2021-02-04,PGANDE TYPE: WEB ONLINE CO: PGANDE,134.91
5,2021-02-04,Yardi Service Ch TYPE: WEB PMTS CO: Yardi Ser...,0.95
9,2021-02-16,To Share 00 REGULAR SAVINGS Mobile Transfer,1600.00
...,...,...,...
287,2022-01-09,Check Card: SQ *TENORI Stanton CA 01/07/22,9.70
288,2022-01-09,Check Card: THE RINKS - ANAHEIM ICE 7145357465...,6.00
289,2022-01-10,Check Card: TARGET 00019364 SANTA ANA CA 01/0...,17.45
290,2022-01-10,Check Card: PANDA EXPRESS #108 SANTA ANA CA 0...,11.27


Splitting the payment descriptions into expenses categories (Food, Items, Activities, Utilities, Stocks, and Transfers)

In [15]:
categories = {
    'Food': ['mcdonald','donut','pho','tea','sandwich','cafe','taco','burger','chicken','panda express','udon','mart','habit','pies',
             'hakata ikkousha','trader','road','woomiok','subway','mexican','bottle','popeyes','pollo','maya','boba','burnstein',
             'ramen','jack in the box','mark','taqueria','market','food','bar','chipotle','brunch', 'costco', 'ralph', 'tokyo central',
             'tenori','asian','restaurant','albertson'],
    'Items': ['brea','target','converse','daiso','michaels','uptown','amazon','maxx','staterbros', 'thrift', 'paypal'],
    'Activities': ['rink','pad','fitness','venmo','uber','spotify','paygo','bowl', 'parking', 'pkng', 'john wayne', 'laverne'],
    'Utilities': ['rapids','brazil','pgande'],
    'Stocks': ['webull'],
    'Transfers': ['transfer']
}

Splitting data values into categories with map function

In [16]:
def desc_to_category(cur_desc):
  lower = cur_desc.lower()
  for i in categories.keys():
    for j in categories[i]:
      if j in lower:
        return i
  return 'Other'

debt_df['Category'] = list(map(desc_to_category, debt_df.Description))
debt_df

,Date,Description,Debits,Category
0,2021-02-01,Check Card: Spotify USA 877-7781161 NY 01/31/21,4.99,Activities
3,2021-02-04,2005LEEABRAZIL-1 TYPE: WEB PMTS CO: 2005LEEAB...,500.00,Utilities
4,2021-02-04,PGANDE TYPE: WEB ONLINE CO: PGANDE,134.91,Utilities
5,2021-02-04,Yardi Service Ch TYPE: WEB PMTS CO: Yardi Ser...,0.95,Other
9,2021-02-16,To Share 00 REGULAR SAVINGS Mobile Transfer,1600.00,Transfers
...,...,...,...,...
287,2022-01-09,Check Card: SQ *TENORI Stanton CA 01/07/22,9.70,Food
288,2022-01-09,Check Card: THE RINKS - ANAHEIM ICE 7145357465...,6.00,Activities
289,2022-01-10,Check Card: TARGET 00019364 SANTA ANA CA 01/0...,17.45,Items
290,2022-01-10,Check Card: PANDA EXPRESS #108 SANTA ANA CA 0...,11.27,Food


Removing 'Transfers' and 'Stocks' because I consider them as other assets 

In [17]:
debt_filt_df = debt_df.loc[(debt_df.Category != 'Transfers') & (debt_df.Category != 'Stocks')]
debt_filt_df

,Date,Description,Debits,Category
0,2021-02-01,Check Card: Spotify USA 877-7781161 NY 01/31/21,4.99,Activities
3,2021-02-04,2005LEEABRAZIL-1 TYPE: WEB PMTS CO: 2005LEEAB...,500.00,Utilities
4,2021-02-04,PGANDE TYPE: WEB ONLINE CO: PGANDE,134.91,Utilities
5,2021-02-04,Yardi Service Ch TYPE: WEB PMTS CO: Yardi Ser...,0.95,Other
21,2021-03-02,2005LEEABRAZIL-1 TYPE: WEB PMTS CO: 2005LEEAB...,500.00,Utilities
...,...,...,...,...
287,2022-01-09,Check Card: SQ *TENORI Stanton CA 01/07/22,9.70,Food
288,2022-01-09,Check Card: THE RINKS - ANAHEIM ICE 7145357465...,6.00,Activities
289,2022-01-10,Check Card: TARGET 00019364 SANTA ANA CA 01/0...,17.45,Items
290,2022-01-10,Check Card: PANDA EXPRESS #108 SANTA ANA CA 0...,11.27,Food


Changing the dataframe into a csv file

In [18]:
debt_filt_df.to_csv('updated_spendings202102_202202.csv')

Adding a month column for pivoting

In [19]:
debt_filt_df['Month'] = list(map(lambda x: x.month_name(), debt_filt_df.Date))
debt_filt_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Description,Debits,Category,Month
0,2021-02-01,Check Card: Spotify USA 877-7781161 NY 01/31/21,4.99,Activities,February
3,2021-02-04,2005LEEABRAZIL-1 TYPE: WEB PMTS CO: 2005LEEAB...,500.00,Utilities,February
4,2021-02-04,PGANDE TYPE: WEB ONLINE CO: PGANDE,134.91,Utilities,February
5,2021-02-04,Yardi Service Ch TYPE: WEB PMTS CO: Yardi Ser...,0.95,Other,February
21,2021-03-02,2005LEEABRAZIL-1 TYPE: WEB PMTS CO: 2005LEEAB...,500.00,Utilities,March
...,...,...,...,...,...
287,2022-01-09,Check Card: SQ *TENORI Stanton CA 01/07/22,9.70,Food,January
288,2022-01-09,Check Card: THE RINKS - ANAHEIM ICE 7145357465...,6.00,Activities,January
289,2022-01-10,Check Card: TARGET 00019364 SANTA ANA CA 01/0...,17.45,Items,January
290,2022-01-10,Check Card: PANDA EXPRESS #108 SANTA ANA CA 0...,11.27,Food,January


Removing unnecessary columns

In [20]:
month_df = debt_filt_df[['Month', 'Debits', 'Category']]
month_df

,Month,Debits,Category
0,February,4.99,Activities
3,February,500.00,Utilities
4,February,134.91,Utilities
5,February,0.95,Other
21,March,500.00,Utilities
...,...,...,...
287,January,9.70,Food
288,January,6.00,Activities
289,January,17.45,Items
290,January,11.27,Food


In [40]:
total_cat = month_df.groupby('Category').sum()
total_cat = total_cat.reset_index()
total_cat['Month'] = 'All'
total_cat

,Category,Debits,Month
0,Activities,1193.82,All
1,Food,1824.51,All
2,Items,904.48,All
3,Other,1273.30,All
4,Utilities,3176.95,All


In [41]:
month_all_df = pd.concat([month_df, total_cat])
month_all_df

,Month,Debits,Category
0,February,4.99,Activities
3,February,500.00,Utilities
4,February,134.91,Utilities
5,February,0.95,Other
21,March,500.00,Utilities
...,...,...,...
0,All,1193.82,Activities
1,All,1824.51,Food
2,All,904.48,Items
3,All,1273.30,Other


Pivot dataframe to add a "Month" widget on Tableau

In [44]:
col_order = month_df.Month.unique()
col_order = np.append('All', col_order)

month_pivot = pd.pivot_table(month_all_df, values = 'Debits', index = 'Category', columns = 'Month')
month_pivot = month_pivot.reindex(col_order, axis = 1)
month_pivot

Month,All,February,March,April,May,June,July,August,September,October,November,December,January
Category,,,,,,,,,,,,,
Activities,1193.82,4.990,15.996667,9.990,15.651667,78.898000,8.067857,43.656667,15.826667,36.620000,21.990000,17.9900,14.000
Food,1824.51,NaN,18.395000,21.598,16.992667,17.933125,18.450000,15.678889,25.925000,23.465000,34.749000,14.4150,12.410
Items,904.48,NaN,31.190000,10.860,15.190000,25.850000,3.220000,17.980000,27.126667,33.072222,24.950000,21.2775,38.355
Other,1273.30,0.950,13.390000,466.875,0.950000,0.950000,NaN,NaN,50.620000,NaN,32.253333,39.0000,NaN
Utilities,3176.95,317.455,318.417500,138.140,500.000000,308.615000,NaN,NaN,NaN,NaN,13.000000,NaN,NaN


Changing the dataframe into a csv file

In [45]:
month_pivot.to_csv('month_spendings202102_202202.csv')